## Mount drive, unzip data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/fingerprint/data.zip



Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: data/5421_zoom.jpg      
  inflating: data/5421_h_shifted.jpg  
  inflating: data/5423_original.jpg  
  inflating: data/5428_original.jpg  
  inflating: data/5429_zoom.jpg      
  inflating: data/5433_zoom.jpg      
  inflating: data/5433_h_shifted.jpg  
  inflating: data/5435_v_shifted.jpg  
  inflating: data/5438_original.jpg  
  inflating: data/5439_original.jpg  
  inflating: data/5441_h_shifted.jpg  
  inflating: data/5455_v_shifted.jpg  
  inflating: data/5458_h_shifted.jpg  
  inflating: data/5459_zoom.jpg      
  inflating: data/5464_noise.jpg     
  inflating: data/5473_h_shifted.jpg  
  inflating: data/5484_h_shifted.jpg  
  inflating: data/5489_zoom.jpg      
  inflating: data/5498_h_shifted.jpg  
  inflating: data/5504_zoom.jpg      
  inflating: data/5505_noise.jpg     
  inflating: data/5506_h_shifted.jpg  
  inflating: data/5509_original.jpg  
  inflating: data/5510_noise.jpg     
  inflating: data/551

## Dataset class

In [4]:
import random
from PIL import Image

from torch.utils.data import Dataset


def get_img_label(img_fp):
    img_fn = img_fp.split('/')[-1]
    img_label = img_fn.split('_')[0]
    return int(img_label)


class TripletFingerprintDataset(Dataset):
    def __init__(self, imgs_fp, classes, transform=None):
        self.imgs_fp = imgs_fp
        self.classes = classes
        self.img_label_dict = {get_img_label(img_fp): [img_fp] for img_fp in self.imgs_fp}
        self.transform = transform

    def __getitem__(self, idx):
        def generate_triplets(idx):
            anchor_fp = self.imgs_fp[idx]
            anchor_class = get_img_label(anchor_fp)
            pos_fp = random.choice(self.img_label_dict[anchor_class])
            neg_class = random.randint(0, len(self.classes) - 1)
            while neg_class == anchor_class:
                neg_class = random.randint(0, len(self.classes) - 1)
            neg_fp = random.choice(self.img_label_dict[neg_class])
            anchor = Image.open(anchor_fp).convert('L')
            pos = Image.open(pos_fp).convert('L')
            neg = Image.open(neg_fp).convert('L')

            return anchor, pos, neg

        anchor, pos, neg = generate_triplets(idx)
        if self.transform is not None:
            anchor = self.transform(anchor)
            pos = self.transform(pos)
            neg = self.transform(neg)

        return (anchor, pos, neg), []

    def __len__(self):
        return len(self.imgs_fp)


## Network and loss

In [9]:
import torch.nn as nn


class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(nn.Conv2d(1, 32, 3), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, 3), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(64, 128, 3), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2)
                                     )

        self.fc = nn.Sequential(nn.Linear(128 * 10 * 10, 256),
                                nn.PReLU(),
                                nn.Linear(256, 256),
                                nn.PReLU(),
                                nn.Linear(256, 128)
                                )

    def forward(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def get_embedding(self, x):
        return self.forward(x)


class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, anchor, pos, neg):
        anchor_embedding = self.embedding_net(anchor)
        pos_embedding = self.embedding_net(pos)
        neg_embedding = self.embedding_net(neg)
        return anchor_embedding, pos_embedding, neg_embedding

    def get_embedding(self, x):
        return self.embedding_net(x)


In [6]:
import torch.nn as nn
import torch.nn.functional as F


class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin=1.):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean() if size_average else losses.sum()


## Trainer

In [16]:
import torch
import numpy as np


def fit(train_loader, val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[],
        start_epoch=0):
    for epoch in range(0, start_epoch):
        scheduler.step()

    for epoch in range(start_epoch, n_epochs):
        scheduler.step()

        # Train stage
        train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)

        message = 'Epoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch + 1, n_epochs, train_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        val_loss, metrics = test_epoch(val_loader, model, loss_fn, cuda, metrics)
        val_loss /= len(val_loader)

        message += '\nEpoch: {}/{}. Validation set: Average loss: {:.4f}'.format(epoch + 1, n_epochs,
                                                                                 val_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())

        print(message)


def train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics):
    for metric in metrics:
        metric.reset()

    model.train()
    losses = []
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        target = target if len(target) > 0 else None
        if not type(data) in (tuple, list):
            data = (data,)
        if cuda:
            data = tuple(d.cuda() for d in data)
            if target is not None:
                target = target.cuda()

        optimizer.zero_grad()
        outputs = model(*data)

        if type(outputs) not in (tuple, list):
            outputs = (outputs,)

        loss_inputs = outputs
        if target is not None:
            target = (target,)
            loss_inputs += target

        loss_outputs = loss_fn(*loss_inputs)
        loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
        losses.append(loss.item())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        for metric in metrics:
            metric(outputs, target, loss_outputs)

        if batch_idx % log_interval == 0:
            message = 'Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data[0]), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), np.mean(losses))
            for metric in metrics:
                message += '\t{}: {}'.format(metric.name(), metric.value())

            print(message)
            losses = []

    total_loss /= (batch_idx + 1)
    return total_loss, metrics


def test_epoch(val_loader, model, loss_fn, cuda, metrics):
    with torch.no_grad():
        for metric in metrics:
            metric.reset()
        model.eval()
        val_loss = 0
        for batch_idx, (data, target) in enumerate(val_loader):
            target = target if len(target) > 0 else None
            if not type(data) in (tuple, list):
                data = (data,)
            if cuda:
                data = tuple(d.cuda() for d in data)
                if target is not None:
                    target = target.cuda()

            outputs = model(*data)

            if type(outputs) not in (tuple, list):
                outputs = (outputs,)
            loss_inputs = outputs
            if target is not None:
                target = (target,)
                loss_inputs += target

            loss_outputs = loss_fn(*loss_inputs)
            loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
            val_loss += loss.item()

            for metric in metrics:
                metric(outputs, target, loss_outputs)

    return val_loss, metrics


## Train model

In [18]:
import glob
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import torchvision.transforms as transforms


# Device
cuda = torch.cuda.is_available()
device = torch.device('cuda' if cuda else 'cpu')

# Hyperparameters
in_channel = 1
batch_size = 128
learning_rate = 0.001
step_size = 50
num_epochs = 30


# Load Data
img_dir = sorted(glob.glob('data/*.jpg'))
classes = [int(i) for i in range(get_img_label(img_dir[-1]))]

transforms = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406],
    #                      std=[0.229, 0.224, 0.225])
])

dataset = TripletFingerprintDataset(img_dir, classes, transform=transforms)
lengths = [int(len(dataset)*0.8), int(len(dataset)*0.2)]
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), len(dataset)-int(len(dataset)*0.8)])

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)

model.to(device)

# Loss and Optimizer
margin = 1.
loss = TripletLoss(margin)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.1)

# Train Network
log_interval = 100
fit(train_loader, test_loader, model, loss, optimizer, scheduler, num_epochs, cuda, log_interval)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/22367 (0%)]	Loss: 0.999804
Train: [12800/22367 (57%)]	Loss: 0.052035
Epoch: 1/30. Train set: Average loss: 0.0426
Epoch: 1/30. Validation set: Average loss: 0.0186
Train: [0/22367 (0%)]	Loss: 0.002752
Train: [12800/22367 (57%)]	Loss: 0.011326
Epoch: 2/30. Train set: Average loss: 0.0130
Epoch: 2/30. Validation set: Average loss: 0.0642
Train: [0/22367 (0%)]	Loss: 0.076678
Train: [12800/22367 (57%)]	Loss: 0.022028
Epoch: 3/30. Train set: Average loss: 0.0163
Epoch: 3/30. Validation set: Average loss: 0.0183
Train: [0/22367 (0%)]	Loss: 0.025353
Train: [12800/22367 (57%)]	Loss: 0.010322
Epoch: 4/30. Train set: Average loss: 0.0080
Epoch: 4/30. Validation set: Average loss: 0.0134
Train: [0/22367 (0%)]	Loss: 0.002688
Train: [12800/22367 (57%)]	Loss: 0.021003
Epoch: 5/30. Train set: Average loss: 0.0162
Epoch: 5/30. Validation set: Average loss: 0.0134
Train: [0/22367 (0%)]	Loss: 0.000000
Train: [12800/22367 (57%)]	Loss: 0.010396
Epoch: 6/30. Train set: Average loss: 0.0070
Epoch:

In [19]:
torch.save(model.state_dict(), '/content/model.pth')